In [1]:
from pyspark.sql import SparkSession,Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import *

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import pandas as pd

In [2]:
# sparknlp.start()
spark = SparkSession.builder.appName('classification').getOrCreate()
# SparkContext.setSystemProperty('spark.executor.memory', '4g')
# SparkContext.setSystemProperty('spark.driver.memory', '4g')
# SparkContext.setSystemProperty('spark.driver.maxResultSize', '3g')
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.dataproc.sql.parquet.enableFooterCache', 'true'),
 ('spark.sql.warehouse.dir', 'file:/spark-warehouse'),
 ('spark.dataproc.sql.joinConditionReorder.enabled', 'true'),
 ('spark.executor.memory', '12859m'),
 ('spark.yarn.am.memory', '640m'),
 ('spark.executor.cores', '4'),
 ('spark.dataproc.sql.local.rank.pushdown.enabled', 'true'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://cluster-39d1-m:8088/proxy/application_1678082267476_0005'),
 ('spark.executor.instances', '2'),
 ('spark.app.startTime', '1678085441157'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.yarn.unmanagedAM.enabled', 'true'),
 ('spark.history.fs.logDirectory',
  'gs://dataproc-temp-us-central1-635155370842-uzamlpgc/0683fb4b-e3ba-49e7-8c10-185ed2f701af/spark-job-history'),
 ('spark.driver.memory', '16384m'),
 ('spark.submit.deployMode', 'client'),
 ('spark.extra

In [ ]:
# spark = SparkSession.builder.appName('CitationNetworkAnalysis').master('local').config("spark.executor.memory", "3g").getOrCreate()
# spark.sparkContext.getConf().getAll()

## Importing parquet dataset

In [6]:
# Import Data
df = spark.read.parquet("gs://msca-bdp-student-gcs/Group5_Final_Project/raw_data/clean_data_march5_nopartition.parquet")

In [3]:
# raw_data = "gs://msca-bdp-student-gcs/Group5_Final_Project/raw_data/dblp_v14.json"
# df = spark.read.json(raw_data)
# df_clean = df.select(df.id, df.fos[0].getItem('name'),
#                      df.doc_type,
#                      df.authors[0].getItem('name'),
#                      df.authors[0].getItem('id'),
#                      df.authors[0].getItem('org'),
#                      df.abstract, df.title, df.year, 
#                      df.n_citation, df.references,
#                      df.keywords)
# df_clean = df_clean.withColumnRenamed("fos[0].name","category")\
#                 .withColumnRenamed("authors[0].name","author_name")\
#                 .withColumnRenamed("authors[0].org","author_org")\
#                 .withColumnRenamed("authors[0].id","author_id")\
#                 .withColumnRenamed("id","paper_id")
# # dropping rows with na in any column
# df_new = df_clean.na.drop()
# df_new = df_new.withColumn("n_references",size(col("references")))
# #removing rows where abstract is empty
# df_read = df_new.filter("abstract != ''")
# df_read = df_read.filter(trim(col("abstract")) != "null")
# #remove all the abstracts which are duplicate
# abstracts_to_remove = df_read.groupBy("abstract").count().orderBy(col("count")).filter(col("count")>1)
# abstracts_remove_list = abstracts_to_remove.rdd.map(lambda x: x[0]).collect()
# df_read = df_read.filter(~df_read.abstract.isin(abstracts_remove_list))
# #trim whitespace
# df_read = df_read.withColumn("abstract", trim(df.abstract))
# #remove redundant space between words
# df_read = df_read.withColumn("abstract", regexp_replace("abstract", "\s+", " "))
# #remove punctuation marks
# df_read = df_read.withColumn("abstract", regexp_replace("abstract", "[^\\w\\s]+", ""))
# df_subset = df_read.filter((df_read['year'] >= 1990) & (df_read['year'] <= 2020))

In [4]:
filtered_df = df_subset.filter((col("year") >= 2013) & (col("year") <= 2019))
filtered_df= filtered_df.select(filtered_df.title, filtered_df.n_citation, filtered_df.abstract, filtered_df.category, filtered_df.year)

In [5]:
filtered_df.printSchema()

root
 |-- title: string (nullable = true)
 |-- n_citation: long (nullable = true)
 |-- abstract: string (nullable = true)
 |-- category: string (nullable = true)
 |-- year: long (nullable = true)



In [18]:
filtered_df.count()

23/03/06 06:13:47 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.0 MiB


1024149

In [34]:
filtered_df.groupBy('category').count().orderBy('count', ascending=False).show(n=50, truncate=False)

23/03/06 06:28:00 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/03/06 06:28:11 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB


+--------------------------------------------+-----+
|category                                    |count|
+--------------------------------------------+-----+
|Computer vision                             |47522|
|Data mining                                 |43641|
|Computer science                            |32474|
|Mathematical optimization                   |28181|
|Discrete mathematics                        |24274|
|Convergence (routing)                       |13982|
|Data science                                |13016|
|Control theory                              |7865 |
|World Wide Web                              |7468 |
|Social psychology                           |6872 |
|Kernel (linear algebra)                     |6364 |
|Econometrics                                |5897 |
|Pattern recognition                         |5880 |
|Population                                  |5848 |
|Information system                          |5842 |
|Graph                                       |

In [6]:
df_filter = filtered_df.filter((filtered_df["category"] == "Computer science") | \
                             (filtered_df["category"] == "Data mining") | \
                             (filtered_df["category"] == "Discrete mathematics") | \
                             (filtered_df["category"] == "Mathematical optimization") | \
                             (filtered_df["category"] == "Computer vision"))

In [7]:
df_filter.groupBy('category').count().orderBy('count', ascending=False).show(n=50)

23/03/06 06:52:48 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/03/06 06:53:02 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB


+--------------------+-----+
|            category|count|
+--------------------+-----+
|     Computer vision|47522|
|         Data mining|43641|
|    Computer science|32474|
|Mathematical opti...|28181|
|Discrete mathematics|24274|
+--------------------+-----+



In [8]:
df_filter = df_filter.select('abstract', 'category')

## Model Pipeline

The pipeline includes three steps:
1. `regexTokenizer`: Tokenization with regular expression
2. `stopwordsRemover`: Remove stop words
3. `countVectors`: Count vectors ("document term vectors")

In [10]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="abstract", outputCol="words", pattern="\\W")
# stop words
# add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

## StringIndexer

StringIndexer encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.
In our case, the label column (Category) will be encoded to label indices, from 0 to 35741; the most frequent label (Computer science) will be indexed as 0

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df_filter)
dataset = pipelineFit.transform(df_filter)
dataset.show(5)

23/03/06 06:54:56 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.0 MiB
23/03/06 06:55:27 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/03/06 06:55:39 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/03/06 06:55:40 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.1 MiB


+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|            abstract|            category|               words|            filtered|            features|label|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|We give an explic...|Discrete mathematics|[we, give, an, ex...|[give, explicit, ...|(10000,[17,111,20...|  4.0|
|A method for iden...|Discrete mathematics|[a, method, for, ...|[method, identify...|(10000,[3,6,13,22...|  4.0|
|Software developm...|    Computer science|[software, develo...|[software, develo...|(10000,[1,5,10,11...|  2.0|
|In recent years s...|         Data mining|[in, recent, year...|[recent, years, a...|(10000,[0,1,3,4,5...|  1.0|
|Accuracy of targe...|    Computer science|[accuracy, of, ta...|[accuracy, target...|(10000,[7,10,14,1...|  2.0|
+--------------------+--------------------+--------------------+--------------------+-----------

## Partition Training & Test sets 

In [12]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)

In [ ]:
# # set seed for reproducibility
# (trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))

## Model Training and Evaluation

### Logistic Regression using Count Vector Features

Our model will make predictions and score on the test set; we then look at the top 10 predictions from the highest probability.

In [ ]:
# trainingData.printSchema()

In [13]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)
# lr = LogisticRegression(maxIter=10, featuresCol = 'features', labelCol='label')
lrModel = lr.fit(trainingData)

23/03/06 06:56:04 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 06:56:18 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 06:56:19 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 06:56:32 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 06:56:32 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/03/06 06:56:32 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/03/06 06:56:32 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 06:56:33 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 06:56:33 WARN org.apache.spark.scheduler.DAGSchedule

In [14]:
predictions = lrModel.transform(testData)

In [15]:
predictions.filter(predictions['prediction'] == 0) \
    .select("abstract","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

23/03/06 06:56:43 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.5 MiB


+------------------------------+---------------+------------------------------+-----+----------+
|                      abstract|       category|                   probability|label|prediction|
+------------------------------+---------------+------------------------------+-----+----------+
|A new enhancement method ba...|Computer vision|[0.9998598727167356,6.63172...|  0.0|       0.0|
|Capturing fine resolution a...|Computer vision|[0.9997841236872907,3.13210...|  0.0|       0.0|
|RGBD cameras such as the Mi...|Computer vision|[0.9997538280576812,9.31501...|  0.0|       0.0|
|In realworld environments t...|Computer vision|[0.9996984502455476,4.58607...|  0.0|       0.0|
|Photogrammetry is a popular...|Computer vision|[0.9996892977681281,7.98593...|  0.0|       0.0|
|Nowadays unmanned ground ve...|Computer vision|[0.9996488822222438,8.39489...|  0.0|       0.0|
|RGBD sensors sensors with R...|Computer vision|[0.9996415816239084,1.46112...|  0.0|       0.0|
|Conventional gaze tracking ..

In [16]:
predictions.select("abstract","category","label","prediction") \
    .show(n = 10, truncate = False)

23/03/06 06:58:02 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.5 MiB


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

23/03/06 06:58:16 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.6 MiB


0.8060057674635692

### Logistic Regression using TF-IDF Features

In [18]:
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])
pipelineFit = pipeline.fit(df_filter)
dataset = pipelineFit.transform(df_filter)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("abstract","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

23/03/06 07:00:30 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.0 MiB
23/03/06 07:00:59 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.0 MiB
23/03/06 07:01:00 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/03/06 07:01:25 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/03/06 07:01:27 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 07:01:42 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 07:01:42 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 07:01:57 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 07:01:57 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary w

+------------------------------+---------------+------------------------------+-----+----------+
|                      abstract|       category|                   probability|label|prediction|
+------------------------------+---------------+------------------------------+-----+----------+
|Capturing fine resolution a...|Computer vision|[0.999815583616366,3.867140...|  0.0|       0.0|
|RGBD sensors sensors with R...|Computer vision|[0.9997907025964753,1.03303...|  0.0|       0.0|
|A new enhancement method ba...|Computer vision|[0.9997876334482079,1.42523...|  0.0|       0.0|
|RGBD cameras such as the Mi...|Computer vision|[0.9997766749256943,1.06314...|  0.0|       0.0|
|Photogrammetry is a popular...|Computer vision|[0.999429188904038,1.163338...|  0.0|       0.0|
|Current markerless registra...|Computer vision|[0.9994062397821193,8.49576...|  0.0|       0.0|
|The quality of the synthesi...|Computer vision|[0.9992780928349491,2.51517...|  0.0|       0.0|
|The accurate location of ey..

In [19]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

23/03/06 07:02:49 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.6 MiB


0.7976696764446968

## Cross-Validation

To tune the hyper-parameters for count vector Logistic Regression

In [ ]:
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
pipelineFit = pipeline.fit(df_filter)
dataset = pipelineFit.transform(df_filter)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)
cvModel = cv.fit(trainingData)
predictions = cvModel.transform(testData)
# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

23/03/06 07:09:45 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.0 MiB
23/03/06 07:10:15 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/03/06 07:10:28 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.9 MiB
23/03/06 07:10:29 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.1 MiB
23/03/06 07:10:44 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.1 MiB
23/03/06 07:10:45 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.1 MiB
23/03/06 07:10:45 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.1 MiB
23/03/06 07:10:46 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.1 MiB
23/03/06 07:10:46 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary w

0.8112449253494058

## Naive Bayes

In [20]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("abstract","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

23/03/06 07:04:41 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.1 MiB
23/03/06 07:05:12 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.1 MiB
23/03/06 07:05:13 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.5 MiB


+------------------------------+--------------------+------------------------------+-----+----------+
|                      abstract|            category|                   probability|label|prediction|
+------------------------------+--------------------+------------------------------+-----+----------+
|Anomaly detection from mult...|     Computer vision|[1.0,1.0071127224201312E-16...|  0.0|       0.0|
|Understanding the nature of...|     Computer vision|[1.0,9.592982443060292E-17,...|  0.0|       0.0|
|Identifying freespace or sa...|     Computer vision|[1.0,9.48773264173804E-17,5...|  0.0|       0.0|
|Barcode localization is an ...|     Computer vision|[1.0,9.224008459054469E-17,...|  0.0|       0.0|
|We propose an enhancement o...|     Computer vision|[1.0,8.621709012534492E-17,...|  0.0|       0.0|
|We present a new and more g...|         Data mining|[1.0,8.487071609422034E-17,...|  1.0|       0.0|
|In this paper the concept o...|     Computer vision|[1.0,7.522499774520346E-17,..

In [21]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

23/03/06 07:05:29 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.6 MiB


0.788895186049028

## Random Forest

In [22]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("abstract","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

23/03/06 07:06:04 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 07:06:05 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.2 MiB
23/03/06 07:06:20 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.3 MiB
23/03/06 07:06:35 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.7 MiB
23/03/06 07:06:50 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.8 MiB
23/03/06 07:06:52 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.9 MiB
23/03/06 07:06:53 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 4.1 MiB
23/03/06 07:06:56 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.6 MiB


+------------------------------+---------------+------------------------------+-----+----------+
|                      abstract|       category|                   probability|label|prediction|
+------------------------------+---------------+------------------------------+-----+----------+
|Salient object detection ai...|Computer vision|[0.47141326316539955,0.1998...|  0.0|       0.0|
|The emergence of new wearab...|Computer vision|[0.47083675370811134,0.2178...|  0.0|       0.0|
|This work addresses the pro...|Computer vision|[0.4589659392062028,0.18924...|  0.0|       0.0|
|This paper proposes a novel...|Computer vision|[0.45227157770955645,0.2231...|  0.0|       0.0|
|Realtime specific 3D object...|Computer vision|[0.44725158063896164,0.2148...|  0.0|       0.0|
|Videos tend to yield a more...|Computer vision|[0.4471504212734164,0.21988...|  0.0|       0.0|
|A twostage head detection s...|Computer vision|[0.44612466916919274,0.2253...|  0.0|       0.0|
|Traditional image object cl..

In [23]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

23/03/06 07:07:10 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.7 MiB


0.31877985711108237